In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('AB_NYC_2019.csv')

In [ ]:
X = data.drop(['id', 'name','host_name', 'last_review', 'reviews_per_month'], axis = 1)
X.dropna(axis = 0, inplace = True)
y = X['price']
X.drop(['price'], axis = 1, inplace = True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)

In [ ]:
X_train.shape

In [ ]:
y_test.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['neighbourhood_group', 'neighbourhood', 'room_type']
for col in cols :
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

In [ ]:
print(tf. __version__)

In [ ]:
import tensorflow.compat.v1 as tf
tf.get_default_graph()

In [ ]:
def embedding_model():
    model = Sequential()
    model.add(Embedding(input_dim = 10, output_dim = 5))
    model.add(Conv1D(5, 1, activation = 'relu'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(5, activation = 'relu'))
    model.add(Dense(1, activation = 'linear'))
    return model

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.model_selection import StratifiedKFold
import math
import numpy as np
from sklearn.metrics import mean_squared_error

oof = np.zeros((len(X_train), 1))
preds = np.zeros((len(X_test), 1))

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2020)

for fold, (trn_idx, test_idx) in enumerate(skf.split(X_train, y_train)):
    trn_X, test_X = X_train[trn_idx], X_train[test_idx]
    trn_y, test_y = y_train.iloc[trn_idx], y_train.iloc[test_idx]
    
    model = embedding_model()
    model.compile(loss = 'mse', metrics = [tf.keras.metrics.RootMeanSquaredError()], optimizer = 'adam')
    model.fit(trn_X, trn_y, batch_size = 128, epochs = 100, validation_data = (test_X, test_y), verbose = 0)
    preds = model.predict(test_X)
    oof[test_idx] += preds
    score = np.sqrt(mean_squared_error(test_y, preds))
    preds += model.predict(X_test) / 5

In [ ]:
print('Train RMSE : ', np.sqrt(mean_squared_error(y_train, oof)))
print('Test RMSE :', np.sqrt(mean_sqaured_error(y_test, preds)))